In [1]:
from sklearn.model_selection import train_test_split

import tensorflow as tf
import numpy as np
import cv2
import glob
import pandas as pd

from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [2]:
n_classes = 8

images = [cv2.imread(file) for class_number in range(1,n_classes+1) for file in glob.glob('data/'+ str(class_number) + '/*.jpg')]
images = np.array(images)

In [3]:
Y = [[class_number] for class_number in range(1,n_classes+1) for i in range(len(glob.glob('data/'+ str(class_number) + '/*.jpg')))]
Y = np.array(Y)

In [4]:
images.shape, Y.shape

((5600, 28, 28, 3), (5600, 1))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(images, Y, test_size=0.2, 
                                                    random_state=42, stratify=Y)

print('Train: ', X_train.shape, y_train.shape)
print('Test: ', X_test.shape, y_test.shape)

y_train = pd.get_dummies(pd.DataFrame(y_train, columns=['label']), columns=['label']).values
y_test = pd.get_dummies(pd.DataFrame(y_test, columns=['label']), columns=['label']).values

y_train.shape, y_test.shape

Train:  (4480, 28, 28, 3) (4480, 1)
Test:  (1120, 28, 28, 3) (1120, 1)


((4480, 8), (1120, 8))

In [6]:
# convert in float and normalize to scale (0, 1)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255.
X_test /= 255.

**LeNet**

In [7]:
def create_weights(name, shape):
    return tf.get_variable(name=name, shape=shape, initializer=tf.initializers.he_normal())
  
  
  
def create_biases(size):
    return tf.Variable(tf.constant(0.1, shape=[size]))



def create_convolutional_layer(input_tensor,
                               num_input_channels, 
                               conv_filter_size,        
                               num_filters,
                               name): 
    
    weights = create_weights(name, shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])

    biases = create_biases(num_filters)

    layer = tf.nn.conv2d(input=input_tensor,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')    

    layer += biases

    layer = tf.nn.max_pool(value=layer,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='SAME')
  
    layer = tf.nn.relu(layer)

    return layer



def create_flatten_layer(layer):
    
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])

    return layer

 

def create_fc_layer(input, 
                    num_inputs,    
                    num_outputs,
                    name,
                    use_bn=True,
                    use_relu=True):
  
    
    weights = create_weights(name, shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    layer = tf.matmul(input, weights) + biases
  
    if use_bn:
        batch_mean, batch_var = tf.nn.moments(layer, [0])
        beta = tf.Variable(tf.zeros([num_outputs]))
        scale = tf.Variable(tf.ones([num_outputs]))
        bn = tf.nn.batch_normalization(layer, batch_mean, batch_var, beta, scale, 0.001)
  
    if use_relu:
        layer = tf.nn.relu(bn)

    return layer

In [8]:
def train(batch_size, epochs, keep_proba):
  
    for j in range(epochs+1): 
           
        for i in range(len(X_train) // batch_size + 1):
      
            if i == len(X_train) // batch_size and len(X_train) % batch_size != 0:
                batch_xs, batch_ys = X_train[i*batch_size:], y_train[i*batch_size:]
            elif i == len(X_train) // batch_size and len(X_train) % batch_size == 0:
                pass
            else:
                batch_xs, batch_ys = X_train[i*batch_size:(i+1)*batch_size], y_train[i*batch_size:(i+1)*batch_size]

            sess.run(train_step, feed_dict={x_image: batch_xs, y: batch_ys, keep_probability: keep_proba})
    
    if j % 10 == 0:
        print('Epoch = {epoch:>4}; Train Accuracy = {train_acc:>5.4}; Test Accuracy = {test_acc:>5.4}'.format(epoch=j,
                                                                                                        train_acc=get_train_accuracy(), 
                                                                                                        test_acc=get_test_accuracy()))

In [10]:
def get_train_accuracy(batch_size=256):
    
    
    list_of_acc = []
  
    for i in range(len(X_train) // batch_size + 1):
        
        if i == len(X_train) // batch_size and len(X_train) % batch_size != 0:
            batch_xs, batch_ys = X_train[i*batch_size:], y_train[i*batch_size:]
        elif i == len(X_train) // batch_size and len(X_train) % batch_size == 0:
            pass
        else:
            batch_xs, batch_ys = X_train[i*batch_size:(i+1)*batch_size], y_train[i*batch_size:(i+1)*batch_size]
  
        tmp_acc = sess.run(accuracy, feed_dict={x_image: batch_xs, y: batch_ys, keep_probability: 1.})
        list_of_acc.append(tmp_acc)
    
    acc = np.mean(list_of_acc)
  
    return acc



def get_test_accuracy(batch_size=len(X_test)):
    
    
    list_of_acc = []
  
    for i in range(len(X_test) // batch_size + 1):
        if i == len(X_test) // batch_size and len(X_test) % batch_size != 0:
            batch_xs, batch_ys = X_test[i*batch_size:], y_test[i*batch_size:]
        elif i == len(X_test) // batch_size and len(X_test) % batch_size == 0:
            pass
        else:
            batch_xs, batch_ys = X_test[i*batch_size:(i+1)*batch_size], y_test[i*batch_size:(i+1)*batch_size]
  
    tmp_acc = sess.run(accuracy, feed_dict={x_image: batch_xs, y: batch_ys, keep_probability: 1.})
    list_of_acc.append(tmp_acc)
    
    acc = np.mean(list_of_acc)
  
    return acc

In [0]:
%%time

tf.reset_default_graph()

# placeholders
x_image = tf.placeholder(tf.float32, [None, 28, 28, 3], name='x')
y = tf.placeholder(tf.float32, [None, 8], name='y')
keep_probability = tf.placeholder(tf.float32, name='keep_proba')

# layers of NN
layer_conv_1 = create_convolutional_layer(x_image, 3, 5, 32, 'w1')

layer_conv_2 = create_convolutional_layer(layer_conv_1, 32, 5, 64, 'w2')

h_avg_pool = tf.nn.avg_pool(layer_conv_2, ksize=[1, 7, 7, 1], strides=[1, 1, 1, 1], padding="VALID")

layer_flat = create_flatten_layer(h_avg_pool)

layer_fc1 = create_fc_layer(layer_flat, layer_flat.get_shape()[1:4].num_elements(), 
                            256, 'w3')

layer_fc1_dpor = tf.nn.dropout(layer_fc1, keep_probability)

logit_conv = create_fc_layer(layer_fc1_dpor, 256, 8, 'w4', False, False)

y_conv = tf.nn.softmax(logit_conv)

# define a error and add optimizer
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit_conv, labels=y))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

# estimate accuracy
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accur')

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

train(batch_size=len(X_train), epochs=500, keep_proba=0.2)


Epoch =    0; Train Accuracy = 0.2257; Test Accuracy = 0.2259
Epoch =   10; Train Accuracy = 0.4805; Test Accuracy = 0.4804
Epoch =   20; Train Accuracy = 0.5972; Test Accuracy = 0.5866
Epoch =   30; Train Accuracy = 0.6968; Test Accuracy = 0.6929
Epoch =   40; Train Accuracy = 0.7812; Test Accuracy = 0.7812
Epoch =   50; Train Accuracy = 0.8175; Test Accuracy = 0.8232
Epoch =   60; Train Accuracy = 0.8516; Test Accuracy = 0.8536
Epoch =   70; Train Accuracy = 0.8859; Test Accuracy = 0.8857
Epoch =   80; Train Accuracy = 0.9108; Test Accuracy = 0.9027
Epoch =   90; Train Accuracy = 0.9323; Test Accuracy = 0.9223
Epoch =  100; Train Accuracy = 0.9455; Test Accuracy = 0.9286
Epoch =  110; Train Accuracy = 0.9564; Test Accuracy = 0.942
Epoch =  120; Train Accuracy = 0.9622; Test Accuracy = 0.9446
Epoch =  130; Train Accuracy = 0.9677; Test Accuracy =  0.95
Epoch =  140; Train Accuracy = 0.9724; Test Accuracy = 0.9509
Epoch =  150; Train Accuracy = 0.9744; Test Accuracy = 0.9598
Epoch =  1

In [0]:
print('Train Accuracy = {}'.format(get_train_accuracy()))

print('Test Accuracy = {}'.format(get_test_accuracy()))

Train Accuracy = 1.0
Test Accuracy = 0.9910714030265808


**Save model**

In [12]:
saver = tf.train.Saver()
saver.save(sess, 'model/Classifier')

'/model/Classifier'